In [ ]:
# This finds the stars in images of V1432 Aql

# Created 2021 Aug. 19 by E.S.

In [54]:
import numpy as np
from astropy.stats import sigma_clipped_stats
from photutils.detection import DAOStarFinder
from photutils.aperture import CircularAperture
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import matplotlib.pyplot as plt
from astropy.io import fits, ascii
import glob
import os

%matplotlib qt

In [19]:
stem_data = "/Users/bandari/Documents/git.repos/kriz_projects/data/2021-08-04/"

In [44]:
# star images
file_names_stars = sorted(glob.glob(stem_data + "V1432_Aql-*fit"))

# darks
file_names_darks = sorted(glob.glob(stem_data + "d*fit"))

# flats
file_names_flats = sorted(glob.glob(stem_data + "flat*fit"))

In [45]:
# make a master dark

cube_dark = np.zeros((len(file_names_darks),512,512))
for num_dark in range(0,len(file_names_darks)):
    data = fits.open(file_names_darks[0])[0].data
    cube_dark[num_dark,:,:] = data
master_dark = np.mean(cube_dark,axis=0)

# make a master flat

cube_flat = np.zeros((len(file_names_flats),512,512))
for num_flat in range(0,len(file_names_flats)):
    data = fits.open(file_names_flats[0])[0].data
    cube_flat[num_flat,:,:] = data
master_flat = np.mean(cube_flat,axis=0)

In [38]:
for num_star_frame in range(0,len(file_names_stars)):

    # get some stats for centroiding
    data = fits.open(file_names_stars[0])[0].data
    mean, median, std = sigma_clipped_stats(data, sigma=3.0)

    # find the stars
    daofind = DAOStarFinder(fwhm=3.0, threshold=5.*std)  
    sources = daofind(data - median)  
    for col in sources.colnames:  
        sources[col].info.format = '%.8g'  # for consistent table output

    # write out
    text_file_name = os.path.basename(file_names_stars).split(".")[-1]+".dat"
    ascii.write(sources, "output_other/"+text_file_name, overwrite=False)

In [42]:
'''
# for plotting
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=4.)
norm = ImageNormalize(stretch=SqrtStretch())
plt.imshow(data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')
apertures.plot(color='blue', lw=1.5, alpha=0.5)
'''